TO DO LIST
  *Preprocess the documents by Anything else you think it's needed
  *Return in output k documents, or all the documents with non-zero similarity with the query when the results are less than k. You must use a heap data structure (you can use Python libraries) for maintaining the top-k documents.

jupyter nbextension enable --py widgetsnbextension --sys-prefix
  *fdsfdsfs
  *dsfdfsf

# Libraries

In [1]:
#%% LIBRARIES
import pandas as pd
import numpy as np

from collections import defaultdict
import functions
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer

from sklearn.metrics.pairwise import cosine_similarity
from heapq import heappush, nlargest
import pickle

from ipywidgets import widgets

In [2]:
import importlib
importlib.reload(functions)

<module 'functions' from 'C:\\Users\\Dusica\\ADMThirdHomework\\functions.py'>

# Step 1: Data

In [3]:
airbnb_data=pd.read_csv("Airbnb_Texas_Rentals.csv",usecols=['average_rate_per_night', 'bedrooms_count', 'city',
       'date_of_listing', 'description', 'latitude', 'longitude', 'title','url'],parse_dates=['date_of_listing'])

In [4]:
airbnb_data.columns

Index(['average_rate_per_night', 'bedrooms_count', 'city', 'date_of_listing',
       'description', 'latitude', 'longitude', 'title', 'url'],
      dtype='object')

# Step 2: Create documents

In [5]:
airbnb_data.head(2)

,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,$27,2,Humble,2016-05-01,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...
1,$149,4,San Antonio,2010-11-01,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...


In [6]:
airbnb_data.shape

(18259, 9)

# Clean data

In [7]:
# Check null values of the dataset
airbnb_data.isnull().sum()
#average_rate_per_night -> replace NAN with 0, convert to int
#bedrooms_count -> There are only 3 records so we decided to replace NAN with a category based on the desciption if it's possible. 
#description, latitude, longitude, title -> replace NAN to 'Unknown'

average_rate_per_night    28
bedrooms_count             3
city                       0
date_of_listing            0
description                2
latitude                  34
longitude                 34
title                      3
url                        0
dtype: int64

In [8]:
airbnb_data=functions.clean(airbnb_data)
airbnb_data.isnull().sum()

average_rate_per_night    0
bedrooms_count            0
city                      0
date_of_listing           0
description               0
latitude                  0
longitude                 0
title                     0
url                       0
dtype: int64

In [9]:
airbnb_data.shape

(11532, 9)

#method is run only once at the beginning to make separate .tsv files
functions.create_tsv_documents(airbnb_data)

# Preprocessing

All the documents were preprocessed by

1) Removing stop words

2) Removing punctuation

3) Stemming

4) Lower case
##### remove non english words__?
##### should we remove numbers__?

# 3.1) Conjunctive query

## 3.1.1) Create your index!

#Building a vocabulary
vocabulary,doc_vocabs=functions.build_vocabulary(airbnb_data)
len(vocabulary) #11717
#called only once
pickle.dump(vocabulary, open("vocabulary.p", "wb"))  # save it 

# Compute an inverted index

#compute an inverted index
inverted_idx=functions.compute_inverted_idx(doc_vocabs,vocabulary)
pickle.dump(inverted_idx, open("inverted_idx.p", "wb"))  # save it 

Hint: Since you do not want to compute the inverted index every time you use the Search Engine, it is worth to think to store it in a separate file and load it in memory when needed.



a beautiful house with garden and beach

In [10]:
vocabulary = pickle.load(open("vocabulary.p", "rb"))
inverted_idx = pickle.load(open("inverted_idx.p", "rb"))

# 3.1.2) Execute the query

In [15]:
doc_id_rs,result_df=functions.search_engine(vocabulary,inverted_idx) #private bedroom internet hot tub
result_df

private bedroom internet hot tub


,title,description,city,url
0,Pool/Hot Tub - 3 Room Suite - Private Bathroom,"Two bedrooms, a sitting room &amp; a private bathroom! (2 real beds, a sofa bed &amp; 3 air mattresses) The entire upstairs of my house! (My wife and I live downstairs) A continental breakfast is included (fresh fruit, cereal, milk, coffee bar, snacks, microwave.)\n25 min to downtown Dallas / 30 minutes to AT&amp;T Stadium\n\nSwim in the pool! Sit and soak in the hot tub (by the glow of Tiki torches). Connect with WiFi &amp; Internet. Enjoy movies and programing, in your private rooms, on DirecTV and NETFLIX.",DeSoto,https://www.airbnb.com/rooms/12791862
1,GREAT RENT HOUSE FOR SUPERBOWL,"SUPERBOWL 2017 DREAM VACATION HOUSE!!!\n4 bedroom, 2 bathroom house for rent. Water front. 30 minute drive to the NRG stadium for the Super bowl. Full access to entire house, including kitchen. We have a house next door so we will be available at all times. \n***pictures show house before it's finished. I will post progress pictures as we finish up the redecorating. ***stay three nights, get the fourth night for free or stay a week for a flat rate of $5000. A REFUNDABLE security deposit of $seven-hundred dollars. \nLand Rover available for $75/hr for driver or $399/day. \n\nAMENITIES\n-access to private pool and hot tub\n-stocked kitchen available for use \n-water front view \n-washer and dryer\n-car available for additional rates \n-internet and cable\n\nAvailable for interaction 24/7\n\nVery spacious. 2 acres of land.",Highlands,https://www.airbnb.com/rooms/16558556
2,"Bay view condo with pool , boat dock sleeps 5-6","The modern newly remodeled Tortuga Bay Bay front Condo with private boat dock offers 2 Bedrooms, (Sleeps up to 6) 2 Full Baths, Resort Pool, Hot Tub, Breathtaking Bayfront Balcony, Full Size-Fully Equipped Kitchen, LCD TVs, Wi-Fi , and so Much More than the Mind Can Conceive!\n\n2 Bedrooms (2-Queen Beds, 2-Twin Beds)\nBay front Balcony, Fully Equipped Kitchen, Washer and Dryer\n3 LCD HDTV\nBroadband Internet and Wi-Fi throughout\nResort Pool and Hot Tub \nPrivate Boat Dock, Outstanding Bay Views",South Padre Island,https://www.airbnb.com/rooms/18813079
3,Estrella del Lago (Star of the Lake) Waterfront Lake House with Hot Tub,"Welcome to the beautiful waterfront lake house with water access! This home known as Estrella Del Lago (Star of the Lake) is a 2 story 3 bedroom, 2 and a half bathroom home situated along the pristine shores of Canyon Lake to provide a relaxing retreat and act as a meeting place for friends and family.\nPull up in your boat or jet skis and drop anchor as the property is located in a quiet area is perfect for swimming, fishing, and kayaking (provided - use at your own risk) on the clear blue water. \nA fully equipped kitchen hosts cooking for the whole family. The home bellies a cozy atmosphere with tall ceilings, comfy beds, furnishings, and relaxing environment. You can enjoy a view from just about every room inside the home and the private covered porches which provide plenty of seating, a Gas Grill, a Firepit, and other outdoor furnishings. The main level features an open concept living with a large deck that is an extension of the home. The living room, dining area, kitchen, and a bedroom are on the main floor. \nIn the lower level you will find a guest area with two bedrooms and a full bathroom which provide plenty of sleeping (2 full size bed bunk beds and Queen bed), a second deck with a hot tub, and fire pit are right off the lower level guest rooms. Other amenities include: Time Warner (Spectrum) cable, High Speed Internet, HBO, Showtime, BluRay DVD Player, Multiple TVs, board games and many more.\nPlease visit our to check rates, availability, and other properties. Gruene, and Canyon Lake are famous for their abundance of country music shows, dinner eateries, theater venues, and shopping. Once visitors have had their fill of this part of the Texas Hill Country, they can take a trip to New Braunfels, Schlitterbahn a s

# 3.2) Conjunctive query & Ranking score 

## 3.2.1) Inverted index 

### Calculation of tf-idf values

tf_idf_dic=functions.calculate_tf_idf(airbnb_data,inverted_idx,vocabulary)

inverted_idx2=functions.compute_inverted_idx2(inverted_idx,vocabulary,tf_idf_dic)

pickle.dump(inverted_idx2, open("inverted_idx2.p", "wb"))  # save it into a file # for future usage it can be just imported 

## 3.2.2) Execute the query

In [16]:
inverted_idx2 = pickle.load(open("inverted_idx2.p", "rb"))

In [17]:
functions.search_engine2(5,vocabulary,inverted_idx,inverted_idx2)
       # sun private room

private bedroom internet hot tub
[544, 2022, 9099, 5806, 12787]


,title,description,city,url,similarity
0,Owner's Special Newly Remodeled Beachfront Condo,"The modern newly remodeled in 2017 Sea Vista Beachfront Condo offers 2 Bedrooms, (Sleeps up to 6) \n\n2 Bedrooms (1-King Bed, 2-Full Beds)\nBeachfront Balcony, Fully Equipped Kitchen, Washer and Dryer\nCentral Air Conditioner\n3 LCD HDTV\nBroadband Internet and Wi-Fi throughout\n2 Resort Pools, 2 Hot Tub and 2 Tennis Courts\nPrivate Beach Access, Gulf and Oceanfront Views\nGated Entry and Elevator",South Padre Island,https://www.airbnb.com/rooms/19565733,0.96
0,"Bay view condo with pool , boat dock sleeps 5-6","The modern newly remodeled Tortuga Bay Bay front Condo with private boat dock offers 2 Bedrooms, (Sleeps up to 6) 2 Full Baths, Resort Pool, Hot Tub, Breathtaking Bayfront Balcony, Full Size-Fully Equipped Kitchen, LCD TVs, Wi-Fi , and so Much More than the Mind Can Conceive!\n\n2 Bedrooms (2-Queen Beds, 2-Twin Beds)\nBay front Balcony, Fully Equipped Kitchen, Washer and Dryer\n3 LCD HDTV\nBroadband Internet and Wi-Fi throughout\nResort Pool and Hot Tub \nPrivate Boat Dock, Outstanding Bay Views",South Padre Island,https://www.airbnb.com/rooms/18813079,0.93
0,GREAT RENT HOUSE FOR SUPERBOWL,"SUPERBOWL 2017 DREAM VACATION HOUSE!!!\n4 bedroom, 2 bathroom house for rent. Water front. 30 minute drive to the NRG stadium for the Super bowl. Full access to entire house, including kitchen. We have a house next door so we will be available at all times. \n***pictures show house before it's finished. I will post progress pictures as we finish up the redecorating. ***stay three nights, get the fourth night for free or stay a week for a flat rate of $5000. A REFUNDABLE security deposit of $seven-hundred dollars. \nLand Rover available for $75/hr for driver or $399/day. \n\nAMENITIES\n-access to private pool and hot tub\n-stocked kitchen available for use \n-water front view \n-washer and dryer\n-car available for additional rates \n-internet and cable\n\nAvailable for interaction 24/7\n\nVery spacious. 2 acres of land.",Highlands,https://www.airbnb.com/rooms/16558556,0.93
0,Estrella del Lago (Star of the Lake) Waterfront Lake House with Hot Tub,"Welcome to the beautiful waterfront lake house with water access! This home known as Estrella Del Lago (Star of the Lake) is a 2 story 3 bedroom, 2 and a half bathroom home situated along the pristine shores of Canyon Lake to provide a relaxing retreat and act as a meeting place for friends and family.\nPull up in your boat or jet skis and drop anchor as the property is located in a quiet area is perfect for swimming, fishing, and kayaking (provided - use at your own risk) on the clear blue water. \nA fully equipped kitchen hosts cooking for the whole family. The home bellies a cozy atmosphere with tall ceilings, comfy beds, furnishings, and relaxing environment. You can enjoy a view from just about every room inside the home and the private covered porches which provide plenty of seating, a Gas Grill, a Firepit, and other outdoor furnishings. The main level features an open concept living with a large deck that is an extension of the home. The living room, dining area, kitchen, and a bedroom are on the main floor. \nIn the lower level you will find a guest area with two bedrooms and a full bathroom which provide plenty of sleeping (2 full size bed bunk beds and Queen bed), a second deck with a hot tub, and fire pit are right off the lower level guest rooms. Other amenities include: Time Warner (Spectrum) cable, High Speed Internet, HBO, Showtime, BluRay DVD Player, Multiple TVs, board games and many more.\nPlease visit our to check rates, availability, and other properties. Gruene, and Canyon Lake are famous for their abundance of country music shows, dinner eateries, theater venues, and shopping. Once visitors have had their fill of this part of the Texas Hill Country, they can take a trip to New Braunfels, Schlitterbahn a sprawling theme park with water rides, nearby live music at the numerous

In [18]:
from collections import OrderedDict

In [19]:
import importlib
importlib.reload(functions)

<module 'functions' from 'C:\\Users\\Dusica\\ADMThirdHomework\\functions.py'>

# 4) Our score

In [ ]:
#doc_id_rs,result_df=functions.search_engine(vocabulary,inverted_idx)# private bedroom internet hot

In [20]:
doc_id_rs

[544, 2022, 9099, 5806, 12787]

In [22]:
def calculate_room_nums(doc_id_rs):
    result_df=pd.DataFrame()
    l=[]
    for i in doc_id_rs:
            #take one file
            df=pd.read_csv('data/doc_'+str(i)+'.tsv',sep='\t',usecols=['bedrooms_count'],encoding='ISO-8859-1')
            result_df=result_df.append(df)
    temp=np.unique(result_df.bedrooms_count.values)
    l=[*temp]
    del result_df
    return l

In [23]:
print("How many bedrooms would you preffer?")

available_room_number=calculate_room_nums(doc_id_rs)
w1=widgets.ToggleButtons(
    options=available_room_number,
    description='# of rooms:',
    disabled=False,
    button_style='info',#, # 'success', 'info', 'warning', 'danger' or ''
      )
w1

How many bedrooms would you preffer?


A Jupyter Widget

In [24]:
print("What price per night would you preffer?")

w2=widgets.FloatText(
    value=150,
    description='Price ($):',
    disabled=False
)
w2

What price per night would you preffer?


A Jupyter Widget

In [25]:
def calculate_score(row):
    temp_rate=0
    temp_beds=0
    if row['average_rate_per_night']in range(0,int((chosen_avg_price/2)+1)):
        temp_rate=0.4
    if row['average_rate_per_night']in range(int(chosen_avg_price/2)+1,int(chosen_avg_price)+1):
        temp_rate=0.2 
    if row['average_rate_per_night'] in range(int(chosen_avg_price),int(chosen_avg_price+101)):
        temp_rate=0.1
    if row['bedrooms_count'] == chosen_no_rooms:
        temp_beds=0.3   
    return temp_rate+temp_beds
#    return np.abs(row.average_rate_per_night-chosen_avg_price)*0.7
#+np.abs(row.average_rate_per_night-chosen_avg_price)*0.7

In [26]:
answers=[]
#process_first_question
chosen_no_rooms=w1.get_interact_value()
answers.append(chosen_no_rooms)
#process_second_question
chosen_avg_price=w2.get_interact_value()
answers.append(chosen_avg_price)
score_lst=[]
#calculate score

#Ranking-Title	Description	City	Url

calc_result_df=pd.DataFrame()
l=[]
for i in doc_id_rs:
    #take one file
    df=pd.read_csv('data/doc_'+str(i)+'.tsv',sep='\t',usecols=['average_rate_per_night','bedrooms_count'],encoding='ISO-8859-1')
    calc_result_df=calc_result_df.append(df)

score_lst=[]
for idx in range(len(calc_result_df)):
    score_lst.append(calculate_score(calc_result_df.iloc[idx]))
    
    

In [27]:
def heapify_tuples(BR_score_tuples):
    heap = []
    for item in BR_score_tuples:
         heappush(heap, item)
    return heap

In [28]:
BR_score_tuples=[]
#result_indices=list(result_df.index)
for idx,val in enumerate(score_lst):
    BR_score_tuples.append((round(val,2),doc_id_rs[idx]))#idx index of row
heapified_tuples=heapify_tuples(BR_score_tuples)

In [29]:
heapified_tuples

[(0, 12787), (0.1, 9099), (0.3, 2022), (0.7, 544), (0.3, 5806)]

In [30]:
sorted_scores=nlargest(len(heapified_tuples),heapified_tuples)

In [31]:
sorted_docs_dic=defaultdict(list)
#counter=1
for tup in sorted_scores:
    sorted_docs_dic[tup[0]].append(tup[1])
#    counter=counter+1
sorted_docs_dic

defaultdict(list, {0: [12787], 0.1: [9099], 0.3: [5806, 2022], 0.7: [544]})

In [32]:
sorted_docs_rank_dic=defaultdict(list)
counter_rank=1
for k,v in sorted_docs_dic.items():
    k=counter_rank
    sorted_docs_rank_dic[k]=v
    counter_rank=counter_rank+1
sorted_docs_rank_dic

defaultdict(list, {1: [544], 2: [5806, 2022], 3: [9099], 4: [12787]})

In [33]:
ranking_dict=OrderedDict()
for k,v in (sorted_docs_rank_dic.items()):
    for list_val in v:
        ranking_dict[list_val]=k

In [34]:
ranking_dict

OrderedDict([(544, 1), (5806, 2), (2022, 2), (9099, 3), (12787, 4)])

In [35]:
df=functions.finalize_output(list(ranking_dict.keys()))
df['ranking']=ranking_dict.values()#(df.index+1)

In [36]:
df[['ranking','title','description','city','url']]
df.set_index('ranking')

,title,description,city,url
ranking,,,,
1,Pool/Hot Tub - 3 Room Suite - Private Bathroom,"Two bedrooms, a sitting room &amp; a private bathroom! (2 real beds, a sofa bed &amp; 3 air mattresses) The entire upstairs of my house! (My wife and I live downstairs) A continental breakfast is included (fresh fruit, cereal, milk, coffee bar, snacks, microwave.)\n25 min to downtown Dallas / 30 minutes to AT&amp;T Stadium\n\nSwim in the pool! Sit and soak in the hot tub (by the glow of Tiki torches). Connect with WiFi &amp; Internet. Enjoy movies and programing, in your private rooms, on DirecTV and NETFLIX.",DeSoto,https://www.airbnb.com/rooms/12791862
2,Estrella del Lago (Star of the Lake) Waterfront Lake House with Hot Tub,"Welcome to the beautiful waterfront lake house with water access! This home known as Estrella Del Lago (Star of the Lake) is a 2 story 3 bedroom, 2 and a half bathroom home situated along the pristine shores of Canyon Lake to provide a relaxing retreat and act as a meeting place for friends and family.\nPull up in your boat or jet skis and drop anchor as the property is located in a quiet area is perfect for swimming, fishing, and kayaking (provided - use at your own risk) on the clear blue water. \nA fully equipped kitchen hosts cooking for the whole family. The home bellies a cozy atmosphere with tall ceilings, comfy beds, furnishings, and relaxing environment. You can enjoy a view from just about every room inside the home and the private covered porches which provide plenty of seating, a Gas Grill, a Firepit, and other outdoor furnishings. The main level features an open concept living with a large deck that is an extension of the home. The living room, dining area, kitchen, and a bedroom are on the main floor. \nIn the lower level you will find a guest area with two bedrooms and a full bathroom which provide plenty of sleeping (2 full size bed bunk beds and Queen bed), a second deck with a hot tub, and fire pit are right off the lower level guest rooms. Other amenities include: Time Warner (Spectrum) cable, High Speed Internet, HBO, Showtime, BluRay DVD Player, Multiple TVs, board games and many more.\nPlease visit our to check rates, availability, and other properties. Gruene, and Canyon Lake are famous for their abundance of country music shows, dinner eateries, theater venues, and shopping. Once visitors have had their fill of this part of the Texas Hill Country, they can take a trip to New Braunfels, Schlitterbahn a sprawling theme park with water rides, nearby live music at the numerous libation stations, McAdoos Seafood Company, and pioneer craft shops. Located just 15 miles from New Braunfels and Gruene,\nJust 5 miles south of Estrella Del Lago, the Whitewater amphitheater has become known as the Live Music Show Capital of south Texas, with 20 live performances featuring more than 50 shows. Shows range from rock n' roll, local music, and country's musics most famous musicians.\nFloating down a river has long been the epic journey, from Lewis and Clark's exploratory sojourn to numerous household pets trying to find their way home. Located five minutes from Estrella Del Lago are numerous river outfitters. Guadalupe River outfitters have a rental fleet of water-dirigibles that usher in leisurely maritime adventures on the lush waters of the Guadalupe River. Inner-tubes cradle the relaxed bodies of slow-paced thrill-seekers as they partake of an epic journey around the famed Horseshoe. Floats can be a 1-to 10-mile odyssey that can last three-eight hours depending on the number of frightened thrill seekers you're able to bribe with stale refreshments. River outfitters can accommodate large groups, and rent by the half-day or full-day consisting of floats that traverse 1- or 10-mile journeys, estimated. As the natural beauty of the surrounding riverside evokes audible gasps and joyful weeping, toob mates are encouraged to sup on their own food and drink.",Canyon Lake,https://www.airbnb.com/rooms/19048300
2,GR

In [ ]:
average_rate_per_night
if it is smaller 1---AKA 0.7 otherwise 0
if it is smaller 1---AKA 0.7 otherwise 0
bedrooms_count
if it is smaller 0---AKA 0.3 otherwise 0
if it is smaller 1---AKA 0.7 otherwise 0

In [ ]:
User chose 0-50 ---120
user chose #of rooms 3
results
55 2 ----15
65 4 -----35
125 5 -----50


less number- +2
same number -0
one bigger +1
twice +2


0-60 25
60-120 50
120-230 75
231 and more 100

25%      60.000000   
50%      117.000000  
75%      228.250000  
max      10000.000000